# Introduction
State notebook purpose here

### Imports
Import libraries and write settings here.

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

In [2]:
%matplotlib inline
import os
import torch

import matplotlib.pyplot as plt
import torchvision.models as models

# from flashtorch.utils import apply_transforms, load_image
# from flashtorch.saliency import Backprop

import dataLoader
from configParser import ConfigParser, getModelName

In [3]:
# CUDA support 
if torch.cuda.is_available():
    torch.cuda.set_device(0) # 0
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print("We are using cuda")

We are using cuda


In [4]:
# params
experimentName = "50_11_hier_phase7"
experiment_index = 0
trial_index=1
experiment_type = "multi-trial"

In [5]:
from configParser import ConfigParser
config_parser = ConfigParser(experimentName)

experimentName = os.path.join(experimentName, experiment_type)
print(experimentName)

50_11_hier_phase7/multi-trial


In [6]:
if experiment_type == "multi-trial" or experiment_type == "selected-multi-trial":
    if experiment_type == "multi-trial":
        paramsIterator = config_parser.getHyperpIter()
    else:
        paramsIterator = config_parser.getHyperpSelectedIter()
        
    for indx, e in enumerate(paramsIterator):
        if indx ==experiment_index:
            experiment_params = e
            break
            
elif experiment_type == "single-trial":
    experiment_params = config_parser.read()
else:
    raise


# experiment_params['normalizeFromResnet']=False
# experiment_params['unsupervisedOnTest']=False
# experiment_params["lambda"]=1

print(experiment_params)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 2650, 'n_epochs': 10000, 'learning_rate': 0.001, 'numOfTrials': 3, 'patience': 200, 'useHeirarchy': True, 'downsample': True, 'downsampleOutput': 350, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'lambda': 0.6, 'unsupervisedOnTest': False, 'fc_layers': 1, 'resnet': '18', 'normalizeFromResnet': True, 'dummy': 0}


In [7]:
datasetManager = dataLoader.datasetManager(experimentName, True)
datasetManager.updateParams(experiment_params)
dataset = datasetManager.getDataset()

Creating dataset...
Loading dataset...


100% (550 of 550) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


Going through image files
Creating dataset... Done.


In [8]:
import CNN

speciesList = dataset.getSpeciesList()
numberOfSpecies = len(speciesList)
genusList = dataset.getGenusList()
numberOfGenus = len(genusList)

trialName = experimentName
if experiment_type != "single-trial":
    trialName = os.path.join(experimentName, getModelName(experiment_params, trial_index))

architecture = {
    "species": numberOfSpecies,
    "genus" : numberOfGenus
}
model = CNN.create_model(architecture, experiment_params)
df, epochs, time_elapsed = CNN.loadModel(model, trialName)
train_loader, validation_loader, test_loader = datasetManager.getLoaders()

Loading saved indices...
file /data/BGNN_data/INHS_cropped/50_11/50_11_hier_phase7/multi-trial/tc0.640000_vc0.160000/trainingIndex.pkl read
trainingIndex.pkl 352
file /data/BGNN_data/INHS_cropped/50_11/50_11_hier_phase7/multi-trial/tc0.640000_vc0.160000/valIndex.pkl read
valIndex.pkl 88
file /data/BGNN_data/INHS_cropped/50_11/50_11_hier_phase7/multi-trial/tc0.640000_vc0.160000/testIndex.pkl read
testIndex.pkl 110
Creating loaders...
Creating loaders... Done.


In [9]:
g_intermediate = None
s_intermediate = None
for batch in train_loader:
    x = batch['image']
    A = model.activations(x)
    g = A["genus_intermediate"].cpu().detach().numpy()
    s = A["downsampled_features"].cpu().detach().numpy()
    if g_intermediate is None:
        g_intermediate = g
        s_intermediate = s
    else:
        g_intermediate = np.concatenate((g_intermediate, g), axis=0)
        s_intermediate = np.concatenate((s_intermediate, s), axis=0)
g_intermediate = g_intermediate.T
s_intermediate = s_intermediate.T
print(g_intermediate.shape)
print(s_intermediate.shape)

N/A% (0 of 550) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

Transfrom images...


100% (550 of 550) |######################| Elapsed Time: 0:00:44 Time:  0:00:44


(200, 352)
(350, 352)


In [10]:
correlation = np.corrcoef(g_intermediate, s_intermediate)
print(correlation.shape)

(550, 550)


In [11]:
import plotly.express as px
fig = px.imshow(correlation[0:g_intermediate.shape[0],g_intermediate.shape[0]:])
fig.show()

# Analysis/Modeling
Do work here

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here